In [394]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=FutureWarning) 

In [395]:
booking_df = pd.read_csv("./bookings.csv", sep=";")
booking_df.head(7)

,Hotel,Is Canceled,Lead Time,arrival full date,Arrival Date Year,Arrival Date Month,Arrival Date Week Number,Arrival Date Day of Month,Stays in Weekend nights,Stays in week nights,...,Adults,Children,Babies,Meal,Country,Reserved Room Type,Assigned room type,customer type,Reservation Status,Reservation status_date
0,Resort Hotel,0,342,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,C,Transient,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015-07-01,2015,July,27,1,0,2,...,2,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-03
5,Resort Hotel,0,14,2015-07-01,2015,July,27,1,0,2,...,2,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-03
6,Resort Hotel,0,0,2015-07-01,2015,July,27,1,0,2,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-03


In [396]:
# Приведите названия колонок к нижнему регистру и замените пробелы на знак нижнего подчеркивания.
new_names = booking_df.columns.str.lower().str.replace(' ', '_')
booking_df.columns = new_names

In [397]:
booking_df['children'] = booking_df['children'].astype('Int64')

In [398]:
booking_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 21 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   hotel                      119390 non-null  object
 1   is_canceled                119390 non-null  int64 
 2   lead_time                  119390 non-null  int64 
 3   arrival_full_date          119390 non-null  object
 4   arrival_date_year          119390 non-null  int64 
 5   arrival_date_month         119390 non-null  object
 6   arrival_date_week_number   119390 non-null  int64 
 7   arrival_date_day_of_month  119390 non-null  int64 
 8   stays_in_weekend_nights    119390 non-null  int64 
 9   stays_in_week_nights       119390 non-null  int64 
 10  stays_total_nights         119390 non-null  int64 
 11  adults                     119390 non-null  int64 
 12  children                   119386 non-null  Int64 
 13  babies                     119390 non-null  

In [399]:
# Пользователи из каких стран совершили наибольшее число успешных бронирований? Укажите топ-5.
booking_df[booking_df['is_canceled'] == 0].groupby('country', as_index=False).aggregate({'is_canceled': 'count'}).sort_values(by='is_canceled', ascending=False).head(5)

,country,is_canceled
125,PRT,21071
57,GBR,9676
54,FRA,8481
50,ESP,6391
42,DEU,6069


In [400]:
# На сколько ночей в среднем бронируют отели разных типов?
booking_df.groupby('hotel', as_index=False).aggregate({'stays_total_nights': 'mean'})


,hotel,stays_total_nights
0,City Hotel,2.978142
1,Resort Hotel,4.318547


In [401]:
# Иногда тип номера, полученного клиентом (assigned_room_type), отличается от изначально забронированного (reserved_room_type). Такое может произойти, например, по причине овербукинга. Сколько подобных наблюдений встретилось в датасете?
booking_df[booking_df['reserved_room_type'] != booking_df['assigned_room_type']].shape[0]

14917

In [402]:
# Проанализируйте даты запланированного прибытия. 
# На какой месяц чаще всего успешно оформляли бронь в 2016? Изменился ли самый популярный месяц в 2017?
popular_month_2016 = booking_df[(booking_df['arrival_date_year'] == 2016) & (booking_df['is_canceled'] == 0)].groupby('arrival_date_month', as_index=False).aggregate({'is_canceled': 'count'}).sort_values(by=['is_canceled'], ascending=False).head(1)
popular_month_2017 = booking_df[(booking_df['arrival_date_year'] == 2017) & (booking_df['is_canceled'] == 0)].groupby('arrival_date_month', as_index=False).aggregate({'is_canceled': 'count'}).sort_values(by=['is_canceled'], ascending=False).head(1)

f'2016: month - {popular_month_2016["arrival_date_month"].values[0]}, number of books: {popular_month_2016["is_canceled"].values[0]}; 2017: month - {popular_month_2017["arrival_date_month"].values[0]}, number of books: {popular_month_2017["is_canceled"].values[0]}'

'2016: month - October, number of books: 3689; 2017: month - May, number of books: 3551'

In [403]:
# Сгруппируйте данные по годам и проверьте, на какой месяц бронирования отеля типа City Hotel отменялись чаще всего в каждый из периодов.
canceled_year = booking_df[(booking_df['hotel'] == 'City Hotel') & (booking_df['is_canceled'] == 1)].groupby(['arrival_date_year', 'arrival_date_month'], as_index=False).aggregate({'is_canceled' : 'sum'})
canceled_year_2015 = canceled_year[canceled_year['arrival_date_year'] == 2015][['is_canceled', 'arrival_date_month']].max()
canceled_year_2016 = canceled_year[canceled_year['arrival_date_year'] == 2016][['is_canceled', 'arrival_date_month']].max()
canceled_year_2017 = canceled_year[canceled_year['arrival_date_year'] == 2017][['is_canceled', 'arrival_date_month']].max()
f"Number of cancels: 2015 - month: {canceled_year_2015[1]}, {canceled_year_2015[0]}; 2016 - month: {canceled_year_2016[1]}, {canceled_year_2016[0]}; 2017 - month: {canceled_year_2017[1]}, {canceled_year_2017[0]}"


'Number of cancels: 2015 - month: September, 1543; 2016 - month: September, 1947; 2017 - month: May, 2217'

In [404]:
# Посмотрите на числовые характеристики трёх переменных: adults, children и babies. Какая из них имеет наибольшее среднее значение?
booking_df[['adults', 'children', 'babies']].describe().loc['mean']

adults      1.856403
children     0.10389
babies      0.007949
Name: mean, dtype: Float64

In [405]:
# Создайте колонку total_kids, объединив children и babies. Отели какого типа в среднем пользуются большей популярностью у клиентов с детьми?
booking_df['total_kids'] = booking_df['children']+booking_df['babies']
booking_df.groupby('hotel', as_index=False).agg({'total_kids': 'mean'})

,hotel,total_kids
0,City Hotel,0.096311
1,Resort Hotel,0.142586


In [424]:
# Создайте переменную has_kids, которая принимает значение True, если клиент при бронировании указал хотя бы одного ребенка (total_kids), и False – в противном случае.
client_num = 11
def kids(x, df):
    if df[x] > 0:
        return True
    else: 
        return False

has_kids = kids(client_num, booking_df['total_kids'])
has_kids


False

In [438]:
# Посчитайте отношение количества ушедших пользователей к общему количеству клиентов, выраженное в процентах (churn rate). Укажите, среди какой группы показатель выше.
gone_w_kids = booking_df[(booking_df['total_kids'] > 0) & (booking_df['reservation_status'] != 'Check-Out')]
gone_w_kids = gone_w_kids.count().values[0]
churn_rate_w_kids = (gone_w_kids/booking_df.shape[0]) * 100
f'{churn_rate_w_kids}%'

'2.7297093558924534%'

In [450]:
gone_wo_kids = booking_df[(booking_df['total_kids'] == 0) & (booking_df['reservation_status'] != 'Check-Out')]
gone_wo_kids = gone_wo_kids.count().values[0]
churn_rate_wo_kids = (gone_wo_kids/booking_df.shape[0]) * 100
f'{churn_rate_wo_kids}%'

'34.30856855683055%'

In [451]:
if gone_w_kids < gone_wo_kids:
    print('Number of person gone with chidren is less than without them')
else:
    print('Number of person gone with chidren is less than without them')

Number of person gone with chidren is less then without them
